# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [2]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Additional imports
from dotenv import find_dotenv
from newsapi import NewsApiClient

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\407491\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Read your api key environment variable
load_dotenv(find_dotenv())
api_key = os.getenv("NEWS_API_KEY")

In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [5]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_everything(
    q="bitcoin OR btc",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# How did the call to newsapi do?
# Print total number of articles returned
print(f"Bitcoin and BTC articles: {bitcoin_headlines['totalResults']}")

# Show first article
bitcoin_headlines["articles"][0]

Bitcoin and BTC articles: 7831


{'source': {'id': None, 'name': 'BeInCrypto'},
 'author': 'Nicholas Pongratz',
 'title': 'Bitcoin Mutual Fund Hits the Market as Crypto ETP Race Heats Up',
 'description': 'Maryland-based ProFunds is launching a publicly available mutual fund connected to the value of bitcoin (BTC).',
 'url': 'https://beincrypto.com/bitcoin-mutual-fund-hits-the-market-as-crypto-etp-race-heats-up/',
 'urlToImage': 'https://assets.entrepreneur.com/providers/be-in-crypto/hero-image-be-in-crypto-378754.jpeg',
 'publishedAt': '2021-07-29T11:45:36Z',
 'content': 'This story originally appeared on be[IN]cryptoMaryland-based ProFunds is launching a publicly available mutual fund connected to the value of bitcoin (BTC).\r\nThe Bitcoin Strategy ProFund invests in b… [+1875 chars]'}

In [36]:
# Fetch the Ethereum news articles
ethereum_headlines = newsapi.get_everything(
    q="ethereum OR eth",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total number of articles returned
print(f"Ethereum and ETH articles: {ethereum_headlines['totalResults']}")

# Show first article
ethereum_headlines["articles"][1]

Ethereum and ETH articles: 3056


{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
 'author': 'Lucas Matney',
 'title': 'Tenderly raises $15.3M to help Ethereum developers ship decentralized apps faster',
 'description': 'Blockchain infrastructure startups are heating up as industry fervor brings more developers and users to a space that still feels extremely young despite a heavy institutional embrace of the crypto space in 2021. The latest crypto startup to court the attenti…',
 'url': 'http://techcrunch.com/2021/07/29/tenderly-raises-15-3m-to-help-ethereum-developers-ship-decentralized-apps-faster/',
 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2021/07/Tenderly-Founding-Team-1.jpg?w=600',
 'publishedAt': '2021-07-29T18:27:16Z',
 'content': 'Blockchain infrastructure startups are heating up as industry fervor brings more developers and users to a space that still feels extremely young despite a heavy institutional embrace of the crypto s… [+2468 chars]'}

In [26]:
# Create function to take in a news "corpus"
# and return a DataFrame with sentiment scores
def create_sentiment_df(news):
    news_sentiment = []
    # Generate sentiment scores for each article
    for article in news["articles"]:
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]

            # Build up sentiment list for DataFrame creation
            news_sentiment.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu})
            
        except AttributeError:
            pass
  
    # Create the news sentiment scores DataFrame
    news_df = pd.DataFrame(news_sentiment)

    # Reorder DataFrame columns to make for better viewing pleasure
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    news_df = news_df[cols]

    return news_df


In [28]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_df = create_sentiment_df(bitcoin_headlines)
bitcoin_df.head()

,date,text,compound,positive,negative,neutral
0,2021-07-29,This story originally appeared on be[IN]crypto...,0.3400,0.079,0.000,0.921
1,2021-08-06,"August\r\n6, 2021\r\n3 min read\r\nThis articl...",-0.3400,0.000,0.066,0.934
2,2021-08-06,When my wife started a little garden in our ur...,-0.3400,0.000,0.061,0.939
3,2021-07-19,"Like Dogecoin devotees, the mayor of Reno, and...",0.6908,0.178,0.000,0.822
4,2021-07-08,Opinions expressed by Entrepreneur contributor...,-0.4497,0.043,0.096,0.861


In [37]:
# Create the Ethereum sentiment scores DataFrame
ethereum_df = create_sentiment_df(ethereum_headlines)
ethereum_df.head()

,date,text,compound,positive,negative,neutral
0,2021-07-16,There are many blockchain platforms competing ...,0.3612,0.075,0.000,0.925
1,2021-07-29,Blockchain infrastructure startups are heating...,-0.2411,0.000,0.061,0.939
2,2021-07-14,While retail investors grew more comfortable b...,0.7264,0.164,0.000,0.836
3,2021-08-05,Cent was founded in 2017 as an ad-free creator...,0.6956,0.190,0.000,0.810
4,2021-07-16,ETHEREUM: THE INFINITE GARDEN is a “feature-le...,0.6486,0.175,0.000,0.825


In [15]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.085383,0.056390,0.033540,0.910050
std,0.396758,0.068063,0.057903,0.082785
min,-0.827100,0.000000,0.000000,0.653000
25%,-0.083575,0.000000,0.000000,0.856750
50%,0.000000,0.044000,0.000000,0.920500
75%,0.386825,0.089500,0.066000,1.000000
max,0.883400,0.276000,0.287000,1.000000


In [38]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.136577,0.057040,0.023990,0.918960
std,0.367553,0.065824,0.049211,0.076559
min,-0.812600,0.000000,0.000000,0.714000
25%,0.000000,0.000000,0.000000,0.873000
50%,0.000000,0.050500,0.000000,0.930000
75%,0.401900,0.090000,0.008000,1.000000
max,0.840200,0.237000,0.249000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?  

**A**: The coin with the highest mean positive score is ***Etherium*** at **0.057040** vs 0.056390.

Q: Which coin had the highest compound score?

**A**: The coin with the highest compound score is ***Bitcoin*** at **0.883400** vs 0.840200.

Q. Which coin had the highest positive score?

**A**: The coin with the highest positive score is ***Bitcoin*** at **0.276000** vs 0.237000.

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [11]:
# Instantiate the lemmatizer
# YOUR CODE HERE!

# Create a list of stopwords
# YOUR CODE HERE!

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [13]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---